# Emergency vs Non-Emergency Vehicle Classification
**Source:** Competition page - [JanataHack: Computer Vision Hackathon](https://datahack.analyticsvidhya.com/contest/janatahack-computer-vision-hackathon/#ProblemStatement)

Fatalities due to traffic delays of emergency vehicles such as ambulance & fire brigade is a huge problem. In daily life, we often see that emergency vehicles face difficulty in passing through traffic. So differentiating a vehicle into an emergency and non emergency category can be an important component in traffic monitoring as well as self drive car systems as reaching on time to their destination is critical for these services.

In this problem, you will be working on classifying vehicle images as either belonging to the emergency vehicle or non-emergency vehicle category. For the same, you are provided with the train and the test dataset. Emergency vehicles usually includes police cars, ambulance and fire brigades.

#### Evaluation Metric

The evaluation metric for this competition is Accuracy.


#### Public and Private split

Note that the test data is further randomly divided into Public (40%) and Private (60%) data. Your initial responses will be checked and scored on the Public data.

#### Sample Image

<img src="sample_images/1.png" width="75%">

#### Result
Validation Set Accuracy: 0.9453<br>
Public Leaderboard Score: 0.9618055556<br>
Private Leaderboard Score: 0.9593301435<br>

# Data Preparation

In [ ]:
import os
import pandas as pd
import shutil
from sklearn.model_selection import train_test_split

## Prepare data directories

In [ ]:
source_dir = 'train/images'
training_file = pd.read_csv('train/train.csv', header=0)

print(training_file.columns)
training_file['label'] = training_file['emergency_or_not'].apply(lambda l: 'emergency' if l == 1 else 'not_emergency')
training_file.head()

train, val = train_test_split(training_file, test_size=0.3, random_state=42, shuffle=True)
print(train.label.value_counts())
print(val.label.value_counts())

In [ ]:
test_file = pd.read_csv('test.csv', header=0)

In [ ]:
# create required directories
os.makedirs('model_images',exist_ok=True)
os.makedirs('model_images/training',exist_ok=True)
os.makedirs('model_images/training/emergency',exist_ok=True)
os.makedirs('model_images/training/not_emergency',exist_ok=True)
os.makedirs('model_images/validation',exist_ok=True)
os.makedirs('model_images/validation/emergency',exist_ok=True)
os.makedirs('model_images/validation/not_emergency',exist_ok=True)
os.makedirs('model_images/testing',exist_ok=True)
os.makedirs('model_images/testing/images',exist_ok=True)

## Move data to created directories

In [ ]:
# copy files to created training directories
for _, img in train.iterrows():
    if img.label == 'emergency':
        shutil.copyfile(os.path.join(source_dir, img.image_names), os.path.join('model_images/training/emergency', img.image_names))
    else:
        shutil.copyfile(os.path.join(source_dir, img.image_names), os.path.join('model_images/training/not_emergency', img.image_names))

# copy files to created validation directories
for _, img in val.iterrows():
    if img.label == 'emergency':
        shutil.copyfile(os.path.join(source_dir, img.image_names), os.path.join('model_images/validation/emergency', img.image_names))
    else:
        shutil.copyfile(os.path.join(source_dir, img.image_names), os.path.join('model_images/validation/not_emergency', img.image_names))

# copy files to created testing directories
for _, img in test_file.iterrows():
    shutil.copyfile(os.path.join(source_dir, img.image_names), os.path.join('model_images/testing/images', img.image_names))

# Train Classification Models

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

In [24]:
import numpy as np
import pandas as pd

In [2]:
validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_directory(
    'model_images/validation/', 
    target_size = (224, 224), 
    batch_size=32, 
    class_mode='binary')

Found 1152 images belonging to 2 classes.
Found 494 images belonging to 2 classes.


In [20]:
# https://stackoverflow.com/questions/52270177/how-to-use-predict-generator-on-new-images-keras/55991598#55991598
test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    'model_images/testing/', 
    target_size = (224, 224),
    batch_size=32, 
    shuffle = False,
    class_mode='binary')

Found 706 images belonging to 1 classes.


In [72]:
train_datagen_aug = ImageDataGenerator(rescale=1./255, zoom_range=0.3, rotation_range=50,
                                   width_shift_range=0.2, height_shift_range=0.2, shear_range=0.2, 
                                   horizontal_flip=True, fill_mode='nearest')
train_generator_aug = train_datagen_aug.flow_from_directory(
    directory='model_images/training/', 
    shuffle=True,
    target_size = (224, 224), 
    batch_size=32, 
    class_mode='binary')

Found 1152 images belonging to 2 classes.


# VGG 16

In [107]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('checkpoints/vgg16_with_aug/', monitor='val_loss', verbose=1, save_best_only=True)

In [108]:
pretrained_model = tf.keras.applications.VGG16(include_top=False, weights='imagenet', pooling='max')
pretrained_model.trainable = True

model = tf.keras.models.Sequential([
    tf.keras.layers.InputLayer(input_shape=(224,224,3)),
    pretrained_model,
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=tf.keras.optimizers.Adam(lr=1e-5),
              metrics=['sparse_categorical_accuracy'])

model.summary()


Model: "sequential_17"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 512)               14714688  
_________________________________________________________________
dense_41 (Dense)             (None, 2)                 1026      
Total params: 14,715,714
Trainable params: 14,715,714
Non-trainable params: 0
_________________________________________________________________


In [109]:
history = model.fit(train_generator_aug, steps_per_epoch=36, epochs=10, validation_data=validation_generator, validation_steps=16, verbose=1, callbacks=[checkpoint])              

Epoch 1/10
36/36 [==============================] - ETA: 0s - loss: 0.8231 - sparse_categorical_accuracy: 0.6198 
Epoch 00001: val_loss improved from inf to 0.59945, saving model to checkpoints/vgg16_with_aug/
INFO:tensorflow:Assets written to: checkpoints/vgg16_with_aug/assets
36/36 [==============================] - 625s 17s/step - loss: 0.8231 - sparse_categorical_accuracy: 0.6198 - val_loss: 0.5995 - val_sparse_categorical_accuracy: 0.6700
Epoch 2/10
36/36 [==============================] - ETA: 0s - loss: 0.4501 - sparse_categorical_accuracy: 0.7960 
Epoch 00002: val_loss improved from 0.59945 to 0.39558, saving model to checkpoints/vgg16_with_aug/
INFO:tensorflow:Assets written to: checkpoints/vgg16_with_aug/assets
36/36 [==============================] - 478s 13s/step - loss: 0.4501 - sparse_categorical_accuracy: 0.7960 - val_loss: 0.3956 - val_sparse_categorical_accuracy: 0.8360
Epoch 3/10
36/36 [==============================] - ETA: 0s - loss: 0.3798 - sparse_categorical_accu

In [110]:
model = tf.keras.models.load_model('checkpoints/vgg16_with_aug/')
pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
predicted_classes = 1 - np.argmax(pred, axis=1)

filenames = [_.split('/')[1] for _ in test_generator.filenames]
results = pd.DataFrame({"image_names":filenames,"emergency_or_not":predicted_classes})

results.to_csv('submissions/vgg16_with_aug.csv', index=None)

23/23 [==============================] - 59s 3s/step


### continue training

In [111]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('checkpoints/vgg16_with_aug/', monitor='val_loss', verbose=1, save_best_only=True)
model = tf.keras.models.load_model('checkpoints/vgg16_with_aug/')
history = model.fit(train_generator_aug, steps_per_epoch=36, epochs=10, validation_data=validation_generator, validation_steps=16, verbose=1, callbacks=[checkpoint])

Epoch 1/10
36/36 [==============================] - ETA: 0s - loss: 0.1904 - sparse_categorical_accuracy: 0.9297 
Epoch 00001: val_loss improved from inf to 0.19549, saving model to checkpoints/vgg16_with_aug/
INFO:tensorflow:Assets written to: checkpoints/vgg16_with_aug/assets
36/36 [==============================] - 449s 12s/step - loss: 0.1904 - sparse_categorical_accuracy: 0.9297 - val_loss: 0.1955 - val_sparse_categorical_accuracy: 0.9190
Epoch 2/10
36/36 [==============================] - ETA: 0s - loss: 0.1748 - sparse_categorical_accuracy: 0.9297 
Epoch 00002: val_loss improved from 0.19549 to 0.19088, saving model to checkpoints/vgg16_with_aug/
INFO:tensorflow:Assets written to: checkpoints/vgg16_with_aug/assets
36/36 [==============================] - 496s 14s/step - loss: 0.1748 - sparse_categorical_accuracy: 0.9297 - val_loss: 0.1909 - val_sparse_categorical_accuracy: 0.9251
Epoch 3/10
36/36 [==============================] - ETA: 0s - loss: 0.1766 - sparse_categorical_accu

In [112]:
model = tf.keras.models.load_model('checkpoints/vgg16_with_aug/')
pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
predicted_classes = 1 - np.argmax(pred, axis=1)

filenames = [_.split('/')[1] for _ in test_generator.filenames]
results = pd.DataFrame({"image_names":filenames,"emergency_or_not":predicted_classes})

results.to_csv('submissions/vgg16_with_aug_20_epoch.csv', index=None)

23/23 [==============================] - 80s 3s/step


In [113]:
checkpoint = tf.keras.callbacks.ModelCheckpoint('checkpoints/vgg16_with_aug/', monitor='val_loss', verbose=1, save_best_only=True)
model = tf.keras.models.load_model('checkpoints/vgg16_with_aug/')
history = model.fit(train_generator_aug, steps_per_epoch=36, epochs=10, validation_data=validation_generator, validation_steps=16, verbose=1, callbacks=[checkpoint])

Epoch 1/10
36/36 [==============================] - ETA: 0s - loss: 0.0873 - sparse_categorical_accuracy: 0.9670 
Epoch 00001: val_loss improved from inf to 0.21804, saving model to checkpoints/vgg16_with_aug/
INFO:tensorflow:Assets written to: checkpoints/vgg16_with_aug/assets
36/36 [==============================] - 584s 16s/step - loss: 0.0873 - sparse_categorical_accuracy: 0.9670 - val_loss: 0.2180 - val_sparse_categorical_accuracy: 0.9170
Epoch 2/10
36/36 [==============================] - ETA: 0s - loss: 0.0892 - sparse_categorical_accuracy: 0.9731 
Epoch 00002: val_loss improved from 0.21804 to 0.20807, saving model to checkpoints/vgg16_with_aug/
INFO:tensorflow:Assets written to: checkpoints/vgg16_with_aug/assets
36/36 [==============================] - 1335s 37s/step - loss: 0.0892 - sparse_categorical_accuracy: 0.9731 - val_loss: 0.2081 - val_sparse_categorical_accuracy: 0.9170
Epoch 3/10
36/36 [==============================] - ETA: 0s - loss: 0.0883 - sparse_categorical_acc

In [114]:
model = tf.keras.models.load_model('checkpoints/vgg16_with_aug/')
pred = model.predict(test_generator, steps=len(test_generator), verbose=1)
predicted_classes = 1 - np.argmax(pred, axis=1)

filenames = [_.split('/')[1] for _ in test_generator.filenames]
results = pd.DataFrame({"image_names":filenames,"emergency_or_not":predicted_classes})

results.to_csv('submissions/vgg16_with_aug_30_epoch.csv', index=None)

23/23 [==============================] - 94s 4s/step
